In [28]:
# We will still do ABAE. I made wrong assumptions. I need a test set -> Coherence cant be my only reference metric.
# Re-run on restaurant to see if I did good.

In [1]:
from main.abae.dataset import PositiveNegativeABAEDataset
from main.abae.model_manager import ABAEManagerConfig, ABAEDefaultManagerFactory
import pandas as pd

# First attempt config.
corpus = "../../output/ds/pre_processed.80k.csv"

config = ABAEManagerConfig(model_name="abae", corpus_file_path=corpus)
dataset = pd.read_csv(config.corpus_file_path)

## Hands on: First attempt

In [2]:
abae_manager = ABAEDefaultManagerFactory().factory_method(config)

Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae\abae.embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae\\abae.embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae\abae.embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae\\abae.embeddings.model', 'datetime': '2025-03-03T17:43:01.937665', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}
D:\PycharmProjects\nlp-course-project\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:


Loading the existing found model as requested in path ./output/abae/abae.aspect_embeddings.model
Creating new model


In [31]:
ds = PositiveNegativeABAEDataset(
    dataset=dataset,
    vocabulary=abae_manager.generator.emb_model.vocabulary(),
    negative_size=config.negative_sample_size,
    max_seq_length=config.max_seq_len
)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/91186 [00:00<?, ?it/s]

Max sequence length calculation in progress...


Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [9]:
abae_manager.train(ds)  # 91k records

713/713 ━━━━━━━━━━━━━━━━━━━━ 206s 288ms/step - loss: 5.2341 - max_margin_loss: 5.2330


(<keras.src.callbacks.history.History at 0x1550da6ac30>,
 <Functional name=functional_3, built=True>)

In [3]:
iteration_model = abae_manager.get_compiled_model()

In [4]:
from evaluation import ABAEEvaluationProcessor

inv_vocab = abae_manager.generator.emb_model.model.wv.index_to_key
processor = ABAEEvaluationProcessor.generate_for_model(iteration_model, inv_vocab)
# We can now call and build a coherence evaluator, or we can do one by hand

In [21]:
coherence_model = processor.c_npmi_coherence_model(top_n=10, ds=dataset['comments'].apply(lambda x: x.split(' ')))

coherence_model.get_coherence()

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<7658 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<10855 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<13210 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<15218 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<16894 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<18444 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Diction

-0.20050169543561938

In [22]:
coherence_model.get_coherence_per_topic()

[-0.1298328076991827,
 -0.3099134062514218,
 -0.1410714048773639,
 -0.4294875341228398,
 -0.18663794390456034,
 -0.3611803321881761,
 -0.40561051166438694,
 -0.06539005266596187,
 -0.3642236789799106,
 -0.12881660694144323,
 0.14022147463393575,
 -0.07558067746964896,
 -0.22434788976145623,
 -0.12515236420625445]

In [20]:
coh = processor.u_mass_coherence_model(top_n=100, ds=dataset['comments'].apply(lambda x: x.split(' ')))
coh.get_coherence()

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

-17.4115041503226

In [17]:
coh.get_coherence_per_topic()

[-0.25131441525433473,
 -19.542568970596935,
 -0.8987464202409412,
 -20.444470965018994,
 -3.5804277357546592,
 -20.017026950192054,
 -19.04357780447795,
 -4.339466989859753,
 -21.5622225938978,
 -20.554169882275417,
 -3.8358616383835153,
 -5.533389397541525,
 -3.6526542464332996,
 -0.25131441525433473]

In [5]:
# Good human judgement is C_V
coh = processor.c_v_coherence_model(top_n=100, ds=dataset['comments'].apply(lambda x: x.split(' ')))
coh.get_coherence()

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<7658 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<10855 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<13210 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<15218 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<16894 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<18444 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Diction

0.46951325597595467

A first score of 0.47 is not too bad considering we trained on a subset of data. <br>
It's also worth noting that by adjusting some hp we might boost this.

For C_V a good range of values, as for a rule of thumb, is 0.5-0.7

In [6]:
coh.get_coherence_per_topic()

[0.6734846093622886,
 0.3842562311689465,
 0.5729853802703821,
 0.5299957364201994,
 0.3558408256143046,
 0.5463975198242397,
 0.6588644232851955,
 0.3928530874981103,
 0.3281340157121037,
 0.6223650022085558,
 0.2825007401357925,
 0.38961207232890294,
 0.46663232916056874,
 0.36926361067377544]